# Multi GPU Test

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/MONAI/blob/master/examples/notebooks/multi_gpu_test.ipynb)

## Setup environment

In [1]:
%pip install -qU "monai[ignite]"

Note: you may need to restart the kernel to use updated packages.


## Setup imports

In [1]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import torch

from monai.config import print_config
from monai.engines import create_multigpu_supervised_trainer
from monai.networks.nets import UNet

print_config()

MONAI version: 0.2.0
Python version: 3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31)  [GCC 7.3.0]
Numpy version: 1.18.1
Pytorch version: 1.6.0

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.1.1
scikit-image version: 0.15.0
Pillow version: 7.2.0
Tensorboard version: 2.1.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Test GPUs

In [3]:
lr = 1e-3

net = UNet(
    dimensions=2,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
)


def fake_loss(y_pred, y):
    return (y_pred[0] + y).sum()


def fake_data_stream():
    while True:
        yield torch.rand((10, 1, 64, 64)), torch.rand((10, 1, 64, 64))

### 1 GPU

In [4]:
opt = torch.optim.Adam(net.parameters(), lr)
trainer = create_multigpu_supervised_trainer(net, opt, fake_loss, [torch.device("cuda:0")])
trainer.run(fake_data_stream(), 2, 2)


State:
	iteration: 4
	epoch: 2
	epoch_length: 2
	max_epochs: 2
	output: 40707.8984375
	batch: <class 'tuple'>
	metrics: <class 'dict'>
	dataloader: <class 'generator'>
	seed: 12

### all GPUs

In [5]:
opt = torch.optim.Adam(net.parameters(), lr)
trainer = create_multigpu_supervised_trainer(net, opt, fake_loss, None)
trainer.run(fake_data_stream(), 2, 2)


State:
	iteration: 4
	epoch: 2
	epoch_length: 2
	max_epochs: 2
	output: 35669.37109375
	batch: <class 'tuple'>
	metrics: <class 'dict'>
	dataloader: <class 'generator'>
	seed: 12

### CPU

In [6]:
opt = torch.optim.Adam(net.parameters(), lr)
trainer = create_multigpu_supervised_trainer(net, opt, fake_loss, [])
trainer.run(fake_data_stream(), 2, 2)

State:
	iteration: 4
	epoch: 2
	epoch_length: 2
	max_epochs: 2
	output: 29662.359375
	batch: <class 'tuple'>
	metrics: <class 'dict'>
	dataloader: <class 'generator'>
	seed: 12